In [ ]:
#Code from Daniel Olds to explore with to make more datasets through pyFAI

In [ ]:
ai_pe.array_from_unit(unit='q_A^-1'))

ai_pe.chiArray()

import numpy.ma as ma
chi = ai_pe.chiArray()
mask_this = ma.masked_inside(chi, 0.2, 0.4)

def mask_chi_range(ai, chi_min, chi_max):
    mask_this = ma.masked_inside(ai.chiArray(), chi_min, chi_max)
    return mask_this.mask

def rad_to_deg(rad):
    return 180.0 / np.pi * rad

def deg_to_rad(deg):
    return deg * np.pi / 180.0

my_im*(mask_chi_range(ai_pe, 0, np.pi))

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def f(i):
    phi_list = np.arange(0, np.pi,.05)
    plt.subplot(121)
    plt.cla()

    plt.plot(q, I_list[i],c=cc[i],label=round(phi_list[i],3))
    plt.legend(loc=0)

    plt.subplot(122)
    plt.cla()

    plt.imshow(mask_list[i])
    
    plt.show()

plt.figure()
cc = make_colormap(len(I_list)+10,use_cmap='plasma')

interact(f, i=widgets.IntSlider(min=0, max=len(I_list)-1, step=1, value=10));